In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用于导出到边缘的AutoML图像目标检测模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_object_detection_export_edge.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_object_detection_export_edge.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来创建图像目标检测模型，并将其导出为边缘模型，使用Google Cloud的AutoML。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中的沙拉类别。这个数据集不需要任何特征工程。在本教程中使用的数据集版本存储在一个公共云存储桶中。训练的模型会预测图像中五种食材类别（沙拉、海鲜、西红柿、烘焙食品、奶酪）的边界框位置和相应类型。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建一个AutoML图像目标检测模型，然后将模型导出为TFLite格式的Edge模型。您还可以使用`gcloud`命令行工具或在Google Cloud控制台上在线创建AutoML模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 从`Model`资源中将`Edge`模型导出到Cloud Storage。
- 在本地下载模型。
- 进行本地预测。

费用

本教程使用了 Google 云计算平台 (GCP) 的计费组件：

* Vertex AI
* 云存储

请了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 来根据您的预计使用量生成费用估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的 GA 版本的 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

### GPU 运行时

*如果有这个选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您在什么笔记本环境中，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得 300 美元的免费信用额可用于您的计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保
Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并且将以 `$` 为前缀的 Python 变量插值到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不可以使用多区域存储桶来进行Vertex训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持信息，请参见[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加直播教程会话，您可能正在使用共享测试账户或项目。为了避免用户在创建的资源之间的名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经通过验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行验证您的账户。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务账户授予项目访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"存储对象管理员"，并选择**存储对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到本地环境。

在下面的单元格中，将您的服务账户密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论您的笔记本环境如何，以下步骤都是必需的。**

本教程旨在使用位于公共云存储桶中的训练数据，以及用于导出训练模型的本地云存储桶。您也可以使用存储在本地云存储桶中的自己的训练数据。

请在下方设置您的云存储桶的名称。桶的名称必须在所有谷歌云项目中全局唯一，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时，请运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入我们的Python环境。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点API服务端点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、流水线和端点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

自动机器学习常量

设置专属于AutoML数据集和训练的常量：

- 数据集模式：告诉“数据集”资源服务数据集的类型是什么。
- 数据标记（注释）模式：告诉“数据集”资源服务数据是如何标记（注释）的。
- 数据集训练模式：告诉“Pipeline”资源服务要为哪种任务（例如，分类）训练模型。

In [ ]:
# Image Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_bounding_box_io_format_1.0.0.yaml"
# Image Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_object_detection_1.0.0.yaml"

教程

现在，您已经准备好开始创建自己的AutoML图像目标检测模型。

## 设置客户端

Vertex 客户端库以客户端/服务器模型运行。在您的一侧（Python 脚本）中，您将创建一个客户端，该客户端向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 用于 `Dataset` 资源的数据集服务。
- 用于 `Model` 资源的模型服务。
- 用于训练的管道服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备好了，训练模型的第一步是创建一个管理的数据集实例，然后将标记的数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择为其指定的可读性强的名称。
 - `metadata_schema_uri`：数据集类型的架构。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
 - `parent`：您的`Database`、`Model`和`Endpoint`资源的Vertex位置根路径。
 - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

一个`operation`对象是Vertex处理长时间运行操作的方式。虽然这一步通常很快，但当您在项目中首次使用它时，由于预配会有较长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，通过调用操作方法：

| 方法          | 描述             |
| ----------- | -------------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。       |
| running()   | 返回操作是否仍在运行的True/False。        |
| done()      | 返回操作是否已完成的True/False。 |
| canceled()  | 返回操作是否已取消的True/False。 |
| cancel()    | 取消操作（这可能需要最多30秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("salads-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

图像的 Vertex `Dataset` 资源对您的数据有一些要求：

- 图像必须存储在云存储桶中。
- 每个图像文件必须是图像格式（PNG、JPEG、BMP 等）。
- 必须在您的云存储桶中存储一个包含每个图像路径和标签的索引文件。
- 索引文件必须是 CSV 或 JSONL 格式。

CSV

对于图像对象检测，CSV索引文件的要求如下：

- 没有标题。
- 第一列是图像的云存储路径。
- 第二列是标签。
- 第三/四列是边界框左上角的坐标。坐标已标准化，介于0和1之间。
- 第五/六/七列不使用，应为0。
- 第八/九列是边界框的右下角。

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的Salads数据集的一个版本，使用一个CSV索引文件。

首先快速查看数据。通过在CSV索引文件中计算行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex 数据集资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并传入以下参数：
 - `name`：您给 `Dataset` 资源的可读名称（例如，沙拉）。
 - `import_configs`：导入配置。

- `import_configs`：包含一个字典的 Python 列表，字典中包含键/值条目：
 - `gcs_sources`：一个包含一个或多个索引文件路径的 URI 列表。
 - `import_schema_uri`：标识标记类型的模式。

`import_data()` 方法返回一个长时间运行的 `operation` 对象。这将需要几分钟的时间来完成。如果您是在现场教程中，这将是一个问问题或休息的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

## 训练模型

现在使用您的 Vertex `Dataset` 资源训练一个 AutoML 图像目标检测模型。要训练模型，请按照以下步骤进行：

1. 为 `Dataset` 资源创建一个 Vertex 训练流水线。
2. 执行流水线以开始训练。

### 创建训练管道

您可能会问，我们用管道做什么？通常在工作（例如培训）具有多个步骤，通常按顺序进行：执行步骤 A，执行步骤 B，依此类推时，您会使用管道。通过将步骤放入管道中，我们可以获得以下好处：

1. 可以用于后续训练工作。
2. 可以容器化并作为批处理作业运行。
3. 可以进行分布式处理。
4. 所有步骤均与同一管道作业相关联，以跟踪进度。

使用此辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：管道作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全合格数据集标识符。
- `schema`：数据集标签（注释）培训架构。
- `task`：描述培训作业要求的字典。

该辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，该方法接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道培训作业的完整规范。

现在让我们更深入地了解构建 `training_pipeline` 规范的 *最小* 要求：

- `display_name`：管道作业的人类可读名称。
- `training_task_definition`：数据集标签（注释）培训架构。
- `training_task_inputs`：描述培训作业要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：仅 Vertex 数据集标识符（非全限定）--这是完全限定标识符的最后部分。
 - `fraction_split`：如果指定，用于培训、测试和验证的数据集百分比。否则，百分比会自动由 AutoML 选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他接受Python（类似JSON）字典的参数不同，`task`字段接受Google的protobuf Struct，这与Python字典非常相似。使用`json_format.ParseDict`方法进行转换。

您需要指定的最小字段是：

- `budget_milli_node_hours`：用于训练模型的最大时间预算（计费），其中1000 = 1小时。对于图像对象检测，预算必须至少为20小时。
- `model_type`：部署模型的类型：
  - `CLOUD_HIGH_ACCURACY_1`：用于部署到Google Cloud并优化准确性。
  - `CLOUD_LOW_LATENCY_1`：用于部署到Google Cloud并优化延迟（响应时间），
  - `MOBILE_TF_HIGH_ACCURACY_1`：用于部署到边缘并优化准确性。
  - `MOBILE_TF_LOW_LATENCY_1`：用于部署到边缘并优化延迟（响应时间）。
  - `MOBILE_TF_VERSATILE_1`：用于部署到边缘并优化延迟和准确性之间的权衡。
- `disable_early_stopping`：True/False，用于让AutoML根据自己的判断提前停止训练或训练至整个预算结束。

最后，通过调用辅助函数`create_pipeline`创建流水线，该函数将返回一个训练流水线对象的实例。

In [ ]:
PIPE_NAME = "salads_pipe-" + TIMESTAMP
MODEL_NAME = "salads_model-" + TIMESTAMP

task = json_format.ParseDict(
    {
        "budget_milli_node_hours": 20000,
        "model_type": "MOBILE_TF_LOW_LATENCY_1",
        "disable_early_stopping": False,
    },
    Value(),
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

获取培训管道的信息

现在仅获取此培训管道实例的管道信息。通过调用作业客户服务的`get_training_pipeline`方法，使用以下参数获取本次作业的作业信息：

- `name`: Vertex完全合格的管道标识符。

当模型训练完成时，管道状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

上述模型的训练可能需要超过30分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的Vertex模型资源标识符，即流水线服务分配给它的标识符。您可以从返回的流水线实例中获取此信息，即字段`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在您的模型已经训练完成，您可以获取一些关于您的模型的信息。

评估模型资源

现在找出模型服务认为您的模型有多好。在训练过程中，数据集的一部分被设置为测试（保留）数据，该数据由管道服务用来评估模型。

### 列出所有切片的评估

使用这个辅助函数`list_model_evaluations`，它接受以下参数：

- `name`：`Model`资源的Vertex完全限定模型标识符。

这个辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估 -- 您可能只有一个，然后打印出评估中每个指标的所有关键名称，并针对一小组（`evaluatedBoundingBoxCount`和`boundingBoxMeanAveragePrecision`）打印出结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("evaluatedBoundingBoxCount", metrics["evaluatedBoundingBoxCount"])
        print(
            "boundingBoxMeanAveragePrecision",
            metrics["boundingBoxMeanAveragePrecision"],
        )

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

将AutoML图像目标检测模型导出为Edge模型，然后您可以将其自定义部署到边缘设备，如手机或物联网设备，或者进行本地下载。使用辅助函数`export_model`将模型导出到Google Cloud，该函数接受以下参数：

- `name`: 资源`Model`的Vertex完全限定标识符。
- `format`: 将模型格式保存为的格式。
- `gcs_dest`: 要将SavedFormat模型工件存储到的Cloud Storage位置。

此函数调用`Model`客户端服务的方法`export_model`，参数如下：

- `name`: 资源`Model`的Vertex完全限定标识符。
- `output_config`: 导出模型的目标信息。
  - `artifact_destination.output_uri_prefix`: 要将SavedFormat模型工件存储到的Cloud Storage位置。
  - `export_format_id`: 将模型格式保存为的格式。对于AutoML图像目标检测:
   - `tf-saved-model`: 用于部署到容器的TensorFlow SavedFormat。
   - `tflite`: 用于部署到边缘或移动设备的TensorFlow Lite。
   - `edgetpu-tflite`: 用于TPU的TensorFlow Lite。
   - `tf-js`: 用于Web客户端的TensorFlow。
   - `coral-ml`: 用于Coral设备。

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`来等待该操作同步完成，这会阻塞直到模型被导出。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/" + "salads"


def export_model(name, format, gcs_dest):
    output_config = {
        "artifact_destination": {"output_uri_prefix": gcs_dest},
        "export_format_id": format,
    }
    response = clients["model"].export_model(name=name, output_config=output_config)
    print("Long running operation:", response.operation.name)
    result = response.result(timeout=1800)
    metadata = response.operation.metadata
    artifact_uri = str(metadata.value).split("\\")[-1][4:-1]
    print("Artifact Uri", artifact_uri)
    return artifact_uri


model_package = export_model(model_to_deploy_id, "tflite", MODEL_DIR)

#### 下载TFLite模型文件

现在您已经导出了您的模型的TFLite版本，您可以测试导出的模型，但是首先需要从云存储中下载它。

In [ ]:
! gsutil ls $model_package
# Download the model artifacts
! gsutil cp -r $model_package tflite

tflite_path = "tflite/model.tflite"

#### 实例化一个TFLite解释器

模型的TFLite版本不是TensorFlow SavedModel格式。您不能直接使用像predict()这样的方法。相反，您需要使用TFLite解释器。您必须首先为TFLite模型设置解释器如下：

- 为TFLite模型实例化一个TFLite解释器。
- 指示解释器为模型分配输入和输出张量。
- 获取有关模型输入和输出张量的详细信息，这对于预测是必要的。

In [ ]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]["shape"]

print("input tensor shape", input_shape)

获取测试项目

您将使用数据集中的任意示例作为测试项目。不必担心这个示例很可能被用来训练模型 — 我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n1
test_item = test_items[0].split(",")[0]

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()
test_image = tf.io.decode_jpeg(content)
print("test image shape", test_image.shape)

test_image = tf.image.resize(test_image, (224, 224))
print("test image shape", test_image.shape, test_image.dtype)

test_image = tf.cast(test_image, dtype=tf.uint8).numpy()

利用TFLite模型进行预测

最后，您可以使用您的TFLite模型进行预测，具体步骤如下：

- 将测试图片转换为单个图片的批次（`np.expand_dims`）
- 将输入张量设置为解释器的单个图片批次 (`data`)。
- 调用解释器。
- 获取预测的softmax概率值（`get_tensor`）。
- 确定哪个标签具有最高概率值（`np.argmax`）。

In [ ]:
import numpy as np

data = np.expand_dims(test_image, axis=0)

interpreter.set_tensor(input_details[0]["index"], data)

interpreter.invoke()

softmax = interpreter.get_tensor(output_details[0]["index"])

label = np.argmax(softmax)

print(label)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的个别资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME